In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data=pd.read_csv("tmdb_5000_movies.csv")
data.head()
data.shape
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [3]:
########데이터 전처리##########
#추천시스템 동작 : 기존에 이미 시청한 영화의 제목을 입력 -> 영화와 비슷한 영화를 추천
#                                                    컨텐츠기반추천
data=data[['id','genres', 'vote_average','vote_count', 'popularity', 
      'title', 'keywords', 'overview']]



"""
필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?
문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)
feature engineering(특성공학, )

xdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)
국어, 영어, 수학, 언어능력(50점만점)   수능점수
80     90   100 ,  40   400점
         ...
100명 학생 점수 data를 수집

파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수

x변수들 -> 모델 -> y예측값(y hat)
90 100 95 -> ???예측

타이타닉 data 
승객명단          성별(파생)
Mrs. 이름...        0
Mr. 이름...           1
Dr. ...

"""

'\n필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?\n문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)\nfeature engineering(특성공학, )\n\nxdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)\n국어, 영어, 수학, 언어능력(50점만점)   수능점수\n80     90   100 ,  40   400점\n         ...\n100명 학생 점수 data를 수집\n\n파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수\n\nx변수들 -> 모델 -> y예측값(y hat)\n90 100 95 -> ???예측\n\n타이타닉 data \n승객명단          성별(파생)\nMrs. 이름...        0\nMr. 이름...           1\nDr. ...\n\n'

In [4]:
#data['vote_count'].quantile(0.9)#1838
#상위 10% 지점에 해당되는 vote_count 컬럼 값이 1838 이다.
tmp_m=data['vote_count'].quantile(0.89)#1683
data['vote_count']>=tmp_m
tmp_data=data.copy()[data['vote_count']>=tmp_m]
tmp_data.shape

m=data['vote_count'].quantile(0.9)
data=data[data['vote_count']>=m]
#vote_count값에 대해 대략 500등 안에 들어가는 데이터를 추출

#data.info()
#수치데이터(연속형(점수,온도), 범주형(성별,혈액형,학점))
#연속형 수치데이터 -> 구간 -> 범주화

In [5]:
data.shape
data.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [6]:
c=data['vote_average'].mean()
c #6.96
m #1838

1838.4000000000015

### Bayesian Estimator

In [7]:
#(WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
def weighted_rating(x, m=m, c=c):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R) + (m/(m+v)*c)
    
data['score']=data.apply(weighted_rating, axis=1)    

## 데이터 전처리
### 장르, 키워드

In [8]:
data.head()
data[['genres', 'keywords']].head()
#리스트 내부에 딕셔너리 구조로 저장

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [9]:
#eval()
listStr='[1,2,3,4,5]'
listStr

'[1,2,3,4,5]'

In [10]:
import ast
listData=ast.literal_eval(listStr)
listData

[1, 2, 3, 4, 5]

In [11]:
data['genres']=data['genres'].apply(ast.literal_eval)

In [12]:
data['keywords']=data['keywords'].apply(ast.literal_eval)

In [13]:
data[['genres', 'keywords']]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
3,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,..."
4,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
...,...,...
4291,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...","[{'id': 1366, 'name': 'shotgun'}, {'id': 13000..."
4300,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...","[{'id': 2052, 'name': 'traitor'}, {'id': 6099,..."
4302,"[{'id': 37, 'name': 'Western'}]","[{'id': 801, 'name': 'bounty hunter'}, {'id': ..."
4337,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[{'id': 422, 'name': 'vietnam veteran'}, {'id'..."


In [14]:
data.iloc[0]['genres']
# 모든 데이터의 장르 이름을 출력
#     genres
# 0  Action
#    Adventure
#      ...
# 1  Adventure
#     ...

data.iloc[1]['keywords']
#apply함수 활용

[{'id': 270, 'name': 'ocean'},
 {'id': 726, 'name': 'drug abuse'},
 {'id': 911, 'name': 'exotic island'},
 {'id': 1319, 'name': 'east india trading company'},
 {'id': 2038, 'name': "love of one's life"},
 {'id': 2052, 'name': 'traitor'},
 {'id': 2580, 'name': 'shipwreck'},
 {'id': 2660, 'name': 'strong woman'},
 {'id': 3799, 'name': 'ship'},
 {'id': 5740, 'name': 'alliance'},
 {'id': 5941, 'name': 'calypso'},
 {'id': 6155, 'name': 'afterlife'},
 {'id': 6211, 'name': 'fighter'},
 {'id': 12988, 'name': 'pirate'},
 {'id': 157186, 'name': 'swashbuckler'},
 {'id': 179430, 'name': 'aftercreditsstinger'}]

In [15]:
def name_to_string(columns):
    for col in columns:
        data[col] = (
            data[col].apply(lambda x: [d['name'] for d in x])
                     .apply(lambda x: " ".join(x))
            )

name_to_string(['genres', 'keywords'])

In [16]:
data[['genres', 'keywords']].head()

,genres,keywords
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...
2,Action Adventure Crime,spy based on novel secret agent sequel mi6 bri...
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...


### 벡터화: 장르(문자열) 숫자로 변환 -> 벡터화 작업  
* **BoW**(Bag of Words, 단어가방): 문서를 구성하는 모든 단어들에 대해 표시하는 방법
* **Corpus**(코퍼스): 말뭉치, 단어(도메인) 사전

문서1: 나는 한국 사람  
문서2: 나는 미국 사람  
문서3: 나는 중국 사람  

위 문서의 corpus = {나는, 한국, 미국, 중국, 사람}  

토지소설 -> 원고 -> 토지 코퍼스: {토지 소설에 등장하는 단어들}
태백산맥 소설 -> 원고 -> 태백산맥 단어들 중에는 토지 코퍼스에 있는 단어도 있지만,  
없는 단어도 있을 것이다(예, '지리산' 단어는 토지에 등장하지 않음 -> OOV라고 함)  

#### BoW
전체 문서{d1, ..., dn}을 구성하는 단어장(vocabulary, {t1, t2, ..., tm})을 만든 후     
각각의 문서(n개)에 대해서 단어장에 있는 각각의 단어가 문서 내에 포함되어 있는 지, 아닌 지를 표시하는 방법.

In [24]:
from sklearn.feature_extraction.text import *
#DictVectorizer: 단어의 빈도 정보가 담긴 딕셔너리로부터 BOW 벡터를 생성
v = DictVectorizer()
D = [{'A': 1, 'B': 2}, {'B':3, 'C': 1}] # 단어의 빈도 정보가 담긴 딕셔너리, 문서1, 2
x = v.fit_transform(D)
x

<2x3 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

### Dense vs. Sparse Matrix
* **Sparse(희소행렬)**: 값이 대부분 0으로 채워진 행렬
* **Dense(밀집행렬)**: 대부분의 값이 0이 아닌 값으로 채워진 행렬

### DictVectorizer()
각 단어의 수를 세어놓은 사전에서 BOW 인코딩 벡터를 만든다.

### CountVectorizer()
문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩 벡터를 만든다.

In [25]:
v = DictVectorizer(sparse=False)
x = v.fit_transform(D)
x

array([[1., 2., 0.],
       [0., 3., 1.]])

In [26]:
v.feature_names_

['A', 'B', 'C']

In [27]:
# {'C': 4, 'D': 2} 딕셔너리를 v변수에 저장된 벡터화 방식으로 변환(bow)해라
v.transform({'C': 4, 'D': 2})

array([[0., 0., 4.]])

In [28]:
# CountVectorizer(): 문서 -> 단어 토큰 리스트 -> 단어 출현 빈도 -> bow 벡터 변환
corpus = [ # 해당 도메인에 대한 관련 문서들의 집합
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'The last document'
]

cv = CountVectorizer()
cv.fit(corpus)
cv.vocabulary_ # 총 10개의 단어가 corpus(도메인)에 존재

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [29]:
cv.transform(corpus).toarray() # 딕셔너리에 있는 VALUE가 인덱스로

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

In [30]:
# 새로운 문서를 입력받아 BOW생성
print(cv.transform(['Something completely new', 'This is the third doc']).toarray())

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 1 1]]


### stop_words: 불용어
자주 등장하지만 실제 의미 분석을 하는 데 기여가 없는 단어들 토큰을 거하는 작업  
* I, my, me, over, 조사, 접미사, 등

In [31]:
cv = CountVectorizer(stop_words=['is', 'the'])
cv.fit(corpus)
print(cv.vocabulary_)
print(cv.transform(corpus).toarray())

{'this': 7, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 6, 'one': 4, 'last': 3}
[[0 1 1 0 0 0 0 1]
 [0 1 0 0 0 2 0 1]
 [1 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 0]]


In [32]:
cv = CountVectorizer(stop_words='english')
cv.fit(corpus)
cv.vocabulary_

{'document': 0, 'second': 1}

#### 미리 만들어진 stop_words 들 확인

In [33]:
cv.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [34]:
cv = CountVectorizer(stop_words='english', analyzer='char') # analyzer와 stop_words의 순서가 상관이 있음
cv.fit(corpus)
cv.vocabulary_

C:\Users\ycho4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\feature_extraction\text.py:497: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


{'t': 14,
 'h': 6,
 'i': 7,
 's': 13,
 ' ': 0,
 'e': 4,
 'f': 5,
 'r': 12,
 'd': 3,
 'o': 11,
 'c': 2,
 'u': 15,
 'm': 9,
 'n': 10,
 'a': 1,
 'l': 8}

In [35]:
cv = CountVectorizer(token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

## n-gram (Word2vec)
* 모노그램: 토큰 하나만 단어, 바이그램: 두개 토큰을 하나의 단어로 사용
* bigram: This is the first document
```python
['This is', 'is the', 'the first', 'first document']
```

In [36]:
cv = CountVectorizer(ngram_range=(2, 2)) # [start, end] 
cv.fit(corpus)
cv.vocabulary_

{'this is': 11,
 'is the': 2,
 'the first': 6,
 'first document': 1,
 'the second': 8,
 'second second': 5,
 'second document': 4,
 'and the': 0,
 'the third': 9,
 'third one': 10,
 'the last': 7,
 'last document': 3}

In [37]:
cv = CountVectorizer(ngram_range=(1, 2), token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

### Frequency(빈도수) 기반
`[min_df, max_df)`


In [38]:
cv = CountVectorizer(min_df=1, max_df=3)
cv.fit(corpus)
cv.vocabulary_

{'this': 8,
 'is': 3,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 7,
 'one': 5,
 'last': 4}

## 장르 벡터화

In [39]:
count_vector = CountVectorizer(ngram_range=(1, 3))
c_vector_genres = count_vector.fit_transform(data['genres']) #fit() transform() 한번에

In [40]:
print(c_vector_genres.toarray())
print(c_vector_genres.shape)

[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(481, 364)


In [41]:
count_vector = CountVectorizer(ngram_range=(1, 3))
count_vector.fit(data['genres'])
len(count_vector.get_feature_names())

364

In [42]:
count_vector.get_feature_names()

['action',
 'action adventure',
 'action adventure animation',
 'action adventure comedy',
 'action adventure crime',
 'action adventure fantasy',
 'action adventure science',
 'action adventure thriller',
 'action adventure war',
 'action adventure western',
 'action comedy',
 'action comedy crime',
 'action comedy drama',
 'action comedy family',
 'action comedy science',
 'action crime',
 'action crime drama',
 'action crime mystery',
 'action crime thriller',
 'action drama',
 'action drama adventure',
 'action drama fantasy',
 'action drama history',
 'action drama horror',
 'action drama mystery',
 'action drama science',
 'action drama thriller',
 'action family',
 'action family fantasy',
 'action family romance',
 'action fantasy',
 'action fantasy comedy',
 'action horror',
 'action mystery',
 'action mystery science',
 'action mystery thriller',
 'action romance',
 'action science',
 'action science fiction',
 'action thriller',
 'action thriller adventure',
 'action thrille

### 코사인 유사도

In [76]:
from sklearn.metrics.pairwise import cosine_similarity

genre_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]
genre_c_sim

array([[  0,  13,  42, ..., 298, 297, 240],
       [ 11,   1, 200, ..., 329, 330, 240],
       [  2, 376, 216, ..., 314, 304, 240],
       ...,
       [478, 187,  12, ..., 326, 327,   0],
       [479, 466, 383, ..., 220, 224,   0],
       [480, 468, 294, ..., 246, 248,   0]], dtype=int64)

In [90]:
# "The Dark Knight Rises" 영화와 코사인 유사도가 가장 높은 영화를 리턴
def get_recommend_movie_list(df, movie_title, top=30):
    target_movie_index = df[df['title'] == movie_title].index.values
    sim_index = genre_c_sim[target_movie_index, :top].reshape(-1)
    sim_index = sim_index[sim_index != target_movie_index]
    res = df.iloc[sim_index].sort_values('score', ascending=False)
    return res
    
    
get_recommend_movie_list(data, movie_title="The Dark Knight Rises")

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
65,155,Drama Action Crime Thriller,8.2,12002,187.322927,The Dark Knight,dc comics crime fighter secret identity scarec...,Batman raises the stakes in his war on crime. ...,8.035690
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ...",7.767228
2760,264644,Drama Thriller,8.1,2757,66.113340,Room,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...,7.645138
351,1422,Drama Thriller Crime,7.9,4339,63.429157,The Departed,undercover boston police friends mafia underco...,"To take down South Boston's Irish Mafia, the p...",7.621146
1850,111,Action Crime Drama Thriller,8.0,2948,70.105981,Scarface,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...,7.601698
4337,103,Crime Drama,8.0,2535,58.845025,Taxi Driver,vietnam veteran taxi obsession drug dealer nig...,A mentally unstable Vietnam War veteran works ...,7.564085
1051,146233,Drama Thriller Crime,7.9,3085,88.496873,Prisoners,pennsylvania kidnapping maze vigilante rural s...,When Keller Dover's daughter and her friend go...,7.550121
828,24,Action Crime,7.7,4949,79.754966,Kill Bill: Vol. 1,japan coma martial arts kung fu underworld yak...,An assassin is shot at the altar by her ruthle...,7.500378
3701,641,Crime Drama,7.9,2443,11.573034,Requiem for a Dream,drug addiction junkie heroin speed diet unsoci...,The hopes and dreams of four ambitious people ...,7.497657
1829,6977,Crime Drama Thriller,7.7,3003,53.645267,No Country for Old Men,texas drug traffic hitman united states–mexico...,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",7.420140


## 아이템 기반 협업 필터링
* 사용자의 행동 패턴을 기반으로 추천
* user-item 형식의 테이블로 변환이 필요하다

In [111]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### 테이블 변환 함수
* pivot_table

In [112]:
ratings_pivot = pd.pivot_table(data, 'rating', index='userId', columns='movieId')
ratings_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# 영화 아이디는 있지만 제목이 없어서 어떤 영화인지 알 수 없음
movies = pd.read_csv("tmdb_5000_movies.csv")
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [110]:
movies.rename(columns={'id':'movieId'}, inplace=True)

In [115]:
print(ratings.columns)
print(movies.columns)

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Index(['budget', 'genres', 'homepage', 'movieId', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count'],
      dtype='object')


### 데이터프레임 병합(merge)

In [117]:
ratings_movies = pd.merge(ratings, movies, on='movieId') # movieId를 기준으로 ratings에 movies를 병합
ratings_movies.head(1)

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296


In [119]:
#userid가 1인 사람의 데이터를 출력
ratings_movies[ratings_movies['userId'] == 2]

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
100,2,62,3.0,835355749,10500000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",NaN,"[{""id"": 305, ""name"": ""moon""}, {""id"": 306, ""nam...",en,2001: A Space Odyssey,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1968-04-10,68700000,149.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,An epic drama of adventure and exploration,2001: A Space Odyssey,7.9,2998
187,2,153,4.0,835355441,4000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,"[{""id"": 213, ""name"": ""upper class""}, {""id"": 92...",en,Lost in Translation,...,"[{""iso_3166_1"": ""JP"", ""name"": ""Japan""}, {""iso_...",2003-08-31,119723856,102.0,"[{""iso_639_1"": ""ja"", ""name"": ""\u65e5\u672c\u8a...",Released,Everyone wants to be found.,Lost in Translation,7.3,1895
316,2,161,3.0,835355493,85000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...",http://www.warnerbros.co.uk/movies/oceans-eleven,"[{""id"": 378, ""name"": ""prison""}, {""id"": 3430, ""...",en,Ocean's Eleven,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-12-07,450717150,116.0,"[{""iso_639_1"": ""it"", ""name"": ""Italiano""}, {""is...",Released,Are you in or out?,Ocean's Eleven,7.2,3783
396,2,165,3.0,835355441,40000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 35, ""...",http://www.backtothefuture.com/movies/backtoth...,"[{""id"": 390, ""name"": ""skateboarding""}, {""id"": ...",en,Back to the Future Part II,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1989-11-20,332000000,108.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Roads? Where we're going, we don't need roads!",Back to the Future Part II,7.4,3829
538,2,168,3.0,835355710,24000000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",NaN,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",en,Star Trek IV: The Voyage Home,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1986-11-25,133000000,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The key to saving the future can only be found...,Star Trek IV: The Voyage Home,6.9,477
581,2,186,3.0,835355664,27000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",NaN,"[{""id"": 441, ""name"": ""assassination""}, {""id"": ...",en,Lucky Number Slevin,...,"[{""iso_3166_1"": ""DE"", ""name"": ""Germany""}, {""is...",2006-02-24,56308881,110.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Wrong Time. Wrong Place. Wrong Number.,Lucky Number Slevin,7.4,1323
629,2,223,1.0,835355749,1288000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""na...",NaN,"[{""id"": 660, ""name"": ""monte carlo""}, {""id"": 81...",en,Rebecca,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1940-04-12,6000000,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The shadow of this woman darkened their love.,Rebecca,7.7,336
738,2,235,3.0,835355664,8000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",NaN,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,Stand by Me,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1986-08-22,52287414,89.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"For some, it's the last real taste of innocenc...",Stand by Me,7.8,1522
816,2,248,3.0,835355896,2900000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",NaN,"[{""id"": 419, ""name"": ""dude""}, {""id"": 947, ""nam...",en,Pocketful of Miracles,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1961-12-18,5000000,136.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,You have to see it to believe it!,Pocketful of Miracles,7.3,25
829,2,253,4.0,835355511,7000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id""

In [128]:
rating_by_id = ratings_movies.pivot_table("rating", index="userId", columns="title").fillna(0)
rating_by_id.shape

(670, 856)

In [130]:
rating_by_id = rating_by_id.transpose()

In [132]:
cosine_similarity(rating_by_id, rating_by_id).shape

(856, 856)